# RoboMaster AI Challenge 1.0.0
Armour localisation/identification with YOLO (You Only Look Once: one-shot object detection algorithm)
> Framework: darknet https://github.com/AlexeyAB/darknet

> Architecture:
* YOLOv3, YOLOv4 - original model focus on superior performance
* YOLOv3-tiny, YOLOv4-tiny - lightweight model suitable for mobile application 

##COMP90082 - Software Project - RM-koala

Team member (alphabetical order):

* Akhtar Kurniawan (Akhtar)
* Che-Hao Chang (Ryan)
* Isaac Pedroza (Isaac)
* Jia Yin (Jia)
* Sejin Kim (Kim)


##Set up

####Mount your Google Drive

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

####Verify GPU and CUDA driver

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
import psutil
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
process = psutil.Process(os.getpid())

print("GPU RAM {}MB".format(gpu.memoryTotal))

!/usr/local/cuda/bin/nvcc --version

##First run set up (Execute this section only for the first run)

####Clone darknet repo & Configure the runtime settings & Make the executable

In [ ]:
%cd content/gdrive/My\ Drive
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make

###File type overview
>* .weight/.conv: Initial weight or trained weight file
* .cfg: Configuration file, defines the network architecture
* .data: Points to the data directory
* .names: Defines the class labels

####Retrieve necessary files

In [ ]:
# weight files
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/trained_weight/rm_koala/yolov4-tiny_armour.weights
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/trained_weight/rm_koala/yolov4-tiny_pose.weights

# loader files
%cd data
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/file_list/rm_armour_train.txt
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/file_list/rm_pose_train.txt

# image data
!mkdir images
%cd images
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/images/training/200each.zip
!unzip 200each.zip
%cd ../..

# configuration files
%cd cfg
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/yolov4-tiny_2class_mod.cfg
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/yolov4-tiny_3class.cfg
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/rm_armour.data
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/rm_armour.names
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/rm_pose.data
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/rm_pose.names
%cd ..

##Armour localisation/identification

####Define image displayer & File uploader

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)

###darknet framework command overview
>Command format
* !./darknet detector test [path to .data file] [path to config] [path to weights] [path to image]

>Command line flags
* -ext_output: output bounding box location
* -thresh: Set confidence threshold for prediction
* -out: Output result as json file
* -map: Show mean average percision over training when there is a validation data set 
* -dont_show: Don't show prediction result in place


In [ ]:
%cd darknet

!chmod +x ./darknet

####Armour localisation

In [ ]:
!./darknet detector test cfg/rm_armour.data cfg/yolov4-tiny_2class_mod.cfg yolov4-tiny_armour.weights data/images/200each/blue_2_frame0060.jpg -dont_show
imShow('predictions.jpg')

####Armour identification

In [ ]:
!./darknet detector test cfg/rm_pose.data cfg/yolov4-tiny_3class.cfg yolov4-tiny_pose.weights data/images/200each/blue_2_frame0060.jpg -dont_show
imShow('predictions.jpg')

####Test your own image

Upload your image

In [ ]:
upload()

Copy the following code into a new code block, replace "YOUR_FILE_NAME_HERE" with your file name (extension included) and execute

Armour localisation
```
!./darknet detector test cfg/rm_armour.data cfg/yolov4-tiny_2class_mod.cfg yolov4-tiny_armour.weights YOUR_FILE_NAME_HERE
imShow('predictions.jpg')
```

Armour identification
```
!./darknet detector test cfg/rm_pose.data cfg/yolov4-tiny_3class.cfg yolov4-tiny_pose.weights YOUR_FILE_NAME_HERE
imShow('predictions.jpg')
```

In [ ]:
!./darknet detector test cfg/rm_armour.data cfg/yolov4-tiny_2class_mod.cfg yolov4-tiny_armour.weights blue_1_frame0532.jpg
imShow('predictions.jpg')